# Step 1 Dev Set Labels

In [ ]:
def step1_dev_set_labels(*, step1_iter, k):
    import json
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedCitation import train_loader, dev_loader
    from ProgressPrinter import ProgressPrinter
    from peft import PeftConfig, IA3Config, TaskType
    from transformers import T5ForConditionalGeneration
    import torch
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    dev = dev_loader(batch_size=2)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    t5.load_adapter(f'User_keq{k}_t5base_step1_iter{step1_iter}', 'taskllm')
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_name="taskllm")

    print(f'*** step1_iter: {step1_iter} ***')
    
    with ProgressPrinter(f'{k} acc (dev)') as printer:
        devgolds = []
        
        for examples, labels in dev:
            greedyrewards = []
            for ex, label in zip(examples, labels):
                with torch.no_grad():
                    targets, guesses = [], []
    
                    embeddings = dev.embed( [ ex['ref1'], ex['ref2'] ] + 
                                            [ v['title'] 
                                              for v in ex['profile']
                                              if v['title'] != ex['title'] 
                                            ])
                    scores = torch.max(embeddings[[0,1],:] @ embeddings[2:,:].T, dim=0).values
                    index = torch.topk(scores, dim=0, k=k).indices.to('cpu')
                    titles = [ f'{ex["profile"][ind]["title"]}' for ind in index.tolist() ]
                    concat_titles = ' and '.join([f'"{v}"' for v in titles])
                    prompt = dev.append_to_title(ex, concat_titles)
                    guess = taskllm.predict([prompt], augment=dev.swap_refs).argmax(dim=1).item()
                    devgolds.append({ 'id': ex['id'], 'output': "[2]" if guess else "[1]" })
                    guesses.append(guess)
                    targets.append(int(label == dev.choices[1]))

            greedyacc = (torch.Tensor(guesses) == torch.Tensor(targets)).float().mean().item()

            printer.addobs(greedyacc)

        printer.print()
        printer.autoprint = False

        with open(f'lamp1u_step1_dev_golds.json', 'w') as jsonfile:
            json.dump({ 'task': 'LaMP_1', 'golds': devgolds }, jsonfile)

step1_dev_set_labels(k=5, step1_iter=1)

*** step1_iter: 1 ***
n       5 acc (dev) (since)      dt
1             1.000 (1.000)  958 ms
2             1.000 (1.000)  1.22 s
4             1.000 (1.000)   1.7 s
8             1.000 (1.000)  2.61 s
16            0.875 (0.750)  4.57 s
32            0.844 (0.812)  9.05 s
64            0.750 (0.656)    17 s
128           0.742 (0.734)    33 s
256           0.723 (0.703)  1.11 m
512           0.727 (0.730)  2.21 m
1024          0.741 (0.756)  4.41 m
1250          0.736 (0.712)  5.38 m


In [1]:
def step2_dev_set_labels(*, step2_iter, step1_iter, k):
    import json
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedCitation import train_loader, dev_loader
    from ProgressPrinter import ProgressPrinter
    from peft import PeftConfig, IA3Config, TaskType
    from transformers import T5ForConditionalGeneration
    import torch
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    dev = dev_loader(batch_size=2)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    t5.load_adapter(f'User_keq{k}_t5base_step1_iter{step1_iter}', 'taskllm')
    t5.load_adapter(f'User_keq{k}_t5base_step2_iter{step2_iter}', 'rhat')
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_name="taskllm")
    rewardpredictor = RewardPredictor(t5=t5, adapter_name="rhat", model_id=f'User_keq{k}_t5base_step2_iter{step2_iter}')

    def reward_augment(inputs):
        import re
        return [ re.sub(r'Ref1: (.*)\nRef2: (.*)\nExtra:',
                        r'Ref1: \2\nRef2: \1\nExtra:',
                        z)
                 for z in inputs ]

    gumbel = torch.distributions.gumbel.Gumbel(0,1)
    def randomized_similarity(ex, nsamples):
        embeddings = dev.embed( [ ex['ref1'], ex['ref2'] ] + 
                                [ v['title'] 
                                  for v in ex['profile']
                                  if v['title'] != ex['title'] 
                                ])
        scores = torch.max(embeddings[[0,1],:] @ embeddings[2:,:].T, dim=0).values
        temperature = scores[0].item() - scores[4].item()
        gumbel_shape = torch.Size([nsamples, scores.shape[0]])
        gumbels = temperature * gumbel.sample(gumbel_shape)
        return torch.unique(torch.topk(scores.unsqueeze(0) + gumbels, dim=1, k=k).indices, sorted=False, dim=0).to('cpu')

    print(f'*** step1_iter: {step1_iter} step2_iter: {step2_iter} ***')
    nvoters = 1
    
    with ProgressPrinter(f'{k} acc (dev)') as printer:
        devgolds = []
        
        for examples, labels in dev:
            greedyrewards = []
            for ex, label in zip(examples, labels):
                with torch.no_grad():
                    randos = randomized_similarity(ex, 64)
                    
                    rhatprompts = []
                    prompts = []
                    for rando in randos:
                        titles = [ f'{ex["profile"][ind]["title"]}' for ind in rando ]
                        concat_titles = ' and '.join([f'"{v}"' for v in titles])
                        prompt = dev.append_to_title(ex, concat_titles)
                        prompts.append(prompt)
                        rhatprompt = f"Title: {ex['title']}\nRef1: {ex['ref1']}\nRef2: {ex['ref2']}\n" + '\n'.join(
                                   [ f"Extra: {t}" for t in titles ])
                        rhatprompts.append(rhatprompt)

                    rhats = rewardpredictor.predict(rhatprompts, augment=reward_augment)
                    voters = torch.topk(rhats, k=min(nvoters, len(rhats)), dim=0).indices.view(-1).to('cpu').tolist()
                    guess = taskllm.predict([ prompts[v] for v in voters ], augment=dev.swap_refs).logsumexp(dim=0, keepdim=True).argmax(dim=1)
                        
                    target = int(label == dev.choices[1])
                    reward = int(guess.item() == target)
                    greedyrewards.append(reward)

                    devgolds.append({ 'id': ex['id'], 'output': "[2]" if guess else "[1]" })

            greedyacc = torch.Tensor(greedyrewards).float().mean().item()

            printer.addobs(greedyacc)

        printer.print()
        printer.autoprint = False

        with open(f'lamp1u_step2_dev_golds.json', 'w') as jsonfile:
            json.dump({ 'task': 'LaMP_1', 'golds': devgolds }, jsonfile)

step2_dev_set_labels(k=5, step1_iter=1, step2_iter='6_hyper')

*** step1_iter: 1 step2_iter: 6_hyper ***
n       5 acc (dev) (since)      dt
1             0.500 (0.500)  1.63 s
2             0.750 (1.000)  2.55 s
4             0.625 (0.500)  4.06 s
8             0.688 (0.750)  7.14 s
16            0.719 (0.750)  14.1 s
32            0.734 (0.750)  28.9 s
64            0.742 (0.750)  57.5 s
128           0.734 (0.727)   1.9 m
256           0.732 (0.730)  3.82 m
512           0.728 (0.723)  7.66 m
1024          0.730 (0.732)  15.3 m
1250          0.733 (0.746)  18.7 m
